In [9]:
import pandas as pd
import json
import warnings
import openai
import os

api_key_path = "C:/Users/rijul/Desktop/Courses/DS 5500 Data Science Capstone Project/openai_api_key.txt"


In [10]:
dataset_name = "cars.csv"

data = pd.read_csv(dataset_name)
data.head()

,Dimensions.Height,Dimensions.Length,Dimensions.Width,Engine Information.Driveline,Engine Information.Engine Type,Engine Information.Hybrid,Engine Information.Number of Forward Gears,Engine Information.Transmission,Fuel Information.City mpg,Fuel Information.Fuel Type,Fuel Information.Highway mpg,Identification.Classification,Identification.ID,Identification.Make,Identification.Model Year,Identification.Year,Engine Information.Engine Statistics.Horsepower,Engine Information.Engine Statistics.Torque
0,140,143,202,All-wheel drive,Audi 3.2L 6 cylinder 250hp 236ft-lbs,True,6,6 Speed Automatic Select Shift,18,Gasoline,25,Automatic transmission,2009 Audi A3 3.2,Audi,2009 Audi A3,2009,250,236
1,140,143,202,Front-wheel drive,Audi 2.0L 4 cylinder 200 hp 207 ft-lbs Turbo,True,6,6 Speed Automatic Select Shift,22,Gasoline,28,Automatic transmission,2009 Audi A3 2.0 T AT,Audi,2009 Audi A3,2009,200,207
2,140,143,202,Front-wheel drive,Audi 2.0L 4 cylinder 200 hp 207 ft-lbs Turbo,True,6,6 Speed Manual,21,Gasoline,30,Manual transmission,2009 Audi A3 2.0 T,Audi,2009 Audi A3,2009,200,207
3,140,143,202,All-wheel drive,Audi 2.0L 4 cylinder 200 hp 207 ft-lbs Turbo,True,6,6 Speed Automatic Select Shift,21,Gasoline,28,Automatic transmission,2009 Audi A3 2.0 T Quattro,Audi,2009 Audi A3,2009,200,207
4,140,143,202,All-wheel drive,Audi 2.0L 4 cylinder 200 hp 207 ft-lbs Turbo,True,6,6 Speed Automatic Select Shift,21,Gasoline,28,Automatic transmission,2009 Audi A3 2.0 T Quattro,Audi,2009 Audi A3,2009,200,207


In [11]:
def get_column_properties(df: pd.DataFrame, n_samples: int = 3) -> list[dict]:
        properties_list = []
        for column in df.columns:
            dtype = df[column].dtype
            properties = {}
            if dtype in [int, float, complex]:
                properties["dtype"] = "number"
                properties["std"] = self.check_type(dtype, df[column].std())
                properties["min"] = self.check_type(dtype, df[column].min())
                properties["max"] = self.check_type(dtype, df[column].max())

            elif dtype == bool:
                properties["dtype"] = "boolean"
            elif dtype == object:
                try:
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore")
                        pd.to_datetime(df[column], errors='raise')
                        properties["dtype"] = "date"
                except ValueError:
                    if df[column].nunique() / len(df[column]) < 0.5:
                        properties["dtype"] = "category"
                    else:
                        properties["dtype"] = "string"
            elif pd.api.types.is_categorical_dtype(df[column]):
                properties["dtype"] = "category"
            elif pd.api.types.is_datetime64_any_dtype(df[column]):
                properties["dtype"] = "date"
            else:
                properties["dtype"] = str(dtype)

            if properties["dtype"] == "date":
                try:
                    properties["min"] = df[column].min()
                    properties["max"] = df[column].max()
                except TypeError:
                    cast_date_col = pd.to_datetime(df[column], errors='coerce')
                    properties["min"] = cast_date_col.min()
                    properties["max"] = cast_date_col.max()
            nunique = df[column].nunique()
            if "samples" not in properties:
                non_null_values = df[column][df[column].notnull()].unique()
                n_samples = min(n_samples, len(non_null_values))
                samples = pd.Series(non_null_values).sample(
                    n_samples, random_state=42).tolist()
                properties["samples"] = samples
            properties["num_unique_values"] = nunique
            properties["semantic_type"] = ""
            properties["description"] = ""
            properties_list.append(
                {"column": column, "properties": properties})

        return properties_list


In [12]:
def json_summary(df):
    
    properties = get_column_properties(df)
    
    summary = {
        "file_name": dataset_name,
        "num_entries": int(df.shape[0]),
        "num_fields": int(df.shape[1]),
        "fields": properties
    }
    
    return summary



summary = json_summary(data)

print(json.dumps(summary, indent=4))


{
    "file_name": "cars.csv",
    "num_entries": 5076,
    "num_fields": 18,
    "fields": [
        {
            "column": "Dimensions.Height",
            "properties": {
                "dtype": "int64",
                "samples": [
                    88,
                    231,
                    167
                ],
                "num_unique_values": 198,
                "semantic_type": "",
                "description": ""
            }
        },
        {
            "column": "Dimensions.Length",
            "properties": {
                "dtype": "int64",
                "samples": [
                    141,
                    155,
                    28
                ],
                "num_unique_values": 208,
                "semantic_type": "",
                "description": ""
            }
        },
        {
            "column": "Dimensions.Width",
            "properties": {
                "dtype": "int64",
                "samples": [
               

In [13]:
api_key = open(api_key_path, 'r').readline()

system_prompt = "You are an experienced data analyst that can annotate datasets. Your instructions are as follows:\ni) ALWAYS generate the name of the dataset and the dataset_description\nii) ALWAYS generate a field description.\niii.) ALWAYS generate a semantic_type (a single word) for each field given its values e.g. company, city, number, supplier, location, gender, longitude, latitude, url, ip address, zip code, email, etc\nYou must return an updated JSON dictionary without any preamble or explanation."

user_prompt = json.dumps(summary)

temperature = 0.7  # How creative the response should be (0 = deterministic, 1 = most creative)
max_tokens = 512  # Maximum length of the response

client = openai.OpenAI(api_key=api_key,)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ],
    temperature=temperature,
    max_tokens=max_tokens,
)



In [14]:
print(response.choices[0].message.content)

{
  "file_name": "cars.csv",
  "dataset_description": "This dataset contains information about cars including dimensions, engine information, fuel information, identification details, and engine statistics.",
  "fields": [
    {
      "column": "Dimensions.Height",
      "properties": {
        "dtype": "int64",
        "samples": [88, 231, 167],
        "num_unique_values": 198,
        "semantic_type": "dimension",
        "description": "Height of the car in inches"
      }
    },
    {
      "column": "Dimensions.Length",
      "properties": {
        "dtype": "int64",
        "samples": [141, 155, 28],
        "num_unique_values": 208,
        "semantic_type": "dimension",
        "description": "Length of the car in inches"
      }
    },
    {
      "column": "Dimensions.Width",
      "properties": {
        "dtype": "int64",
        "samples": [217, 59, 247],
        "num_unique_values": 138,
        "semantic_type": "dimension",
        "description": "Width of the car in inch

In [15]:
system_prompt = "You are an experienced data analyst that can annotate datasets. Your instructions are as follows: 1) Generate a brief text summary of the dataset"

user_prompt = json.dumps(summary)

temperature = 0.7
max_tokens = 256

response_2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ],
    temperature=temperature,
    max_tokens=max_tokens,
)

print(response_2.choices[0].message.content)

The dataset "cars.csv" contains 5076 entries with 18 fields. Some of the key fields include: Dimensions (Height, Length, Width), Engine Information (Driveline, Engine Type, Hybrid, Number of Forward Gears, Transmission), Fuel Information (City mpg, Fuel Type, Highway mpg), Identification (Classification, ID, Make, Model Year, Year), Engine Information (Horsepower, Torque). The dataset provides detailed information about various car specifications such as dimensions, engine details, fuel efficiency, and identification details. The fields contain a mix of data types including integers, categories, boolean, and strings, with varying numbers of unique values and sample data points.


In [32]:
def goal_generator(summary: dict, n=3) -> list[dict]:
    """Generate goals given a summary of data"""
    
    SYSTEM_INSTRUCTIONS = """
        You are a an experienced data analyst who can generate a given number of insightful GOALS about data, when given a summary of the data. The ANALYSES YOU RECOMMEND MUST FOLLOW ANALYSES BEST PRACTICES (e.g., use regression analysis only on data that is appropriate) AND BE MEANINGFUL (e.g., results produced should solve real world problems). Each goal must include a question, an analysis (THE ANALYSIS MUST REFERENCE THE EXACT COLUMN FIELDS FROM THE SUMMARY, e.g., regression of X on Y), and a rationale (JUSTIFICATION FOR WHICH dataset FIELDS ARE USED and what we will learn from the analysis). Each goal MUST mention the exact fields from the dataset summary above
        """
    
    FORMAT_INSTRUCTIONS = """
        THE OUTPUT MUST BE A CODE SNIPPET OF A VALID LIST OF JSON OBJECTS. IT MUST USE THE FOLLOWING FORMAT:
        ```[
            { "index": 0,  "question": "How does Y affect X?", "analysis": "correlation of X and Y", "rationale": "This tells about "} ..
            ]
        ```
        THE OUTPUT SHOULD ONLY USE THE JSON FORMAT ABOVE.
        """
        
    user_prompt = f"""The number of GOALS to generate is {n}. The goals should be based on the data summary below, \n\n .
    {summary} \n\n"""

    user_prompt += f"""\n The generated goals SHOULD BE FOCUSED ON THE INTERESTS AND PERSPECTIVE of an experienced data analyst persona, who is insterested in complex, insightful goals about the data. \n"""

    messages = [
        {"role": "system", "content": SYSTEM_INSTRUCTIONS},
        {"role": "user",
            "content":
            f"{user_prompt}\n\n {FORMAT_INSTRUCTIONS} \n\n. The generated {n} goals are: \n "}]

    result = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
        )

    return result.choices[0].message.content

In [35]:
goals = goal_generator(response.choices[0].message.content)


In [36]:
print(goals)

```json
[
    {
        "index": 0,
        "question": "Is there a relationship between the dimensions of the car (height, length, width) and the type of driveline?",
        "analysis": "ANOVA of Dimensions.Height, Dimensions.Length, Dimensions.Width by Engine Information.Driveline",
        "rationale": "By analyzing the dimensions of the car based on the type of driveline, we can understand if certain driveline types are associated with specific car sizes, providing insights for car manufacturers and buyers."
    },
    {
        "index": 1,
        "question": "What is the distribution of the engine types based on whether the car is a hybrid or not?",
        "analysis": "Frequency distribution of Engine Information.Engine Type by Engine Information.Hybrid",
        "rationale": "Examining how different engine types are distributed among hybrid and non-hybrid cars can help in understanding the prevalence of certain engine technologies in eco-friendly vehicles versus traditional on